In [1]:
import numpy as np
import tensorflow as tf
import json
import os
from enum import unique, Enum
from multiprocessing import Queue, Process

## Deep Neural Network Class
# Abstract class which wraps a Keras model, providing common model functionality
class DNNModel(object):
    def __init__(self, name: str):
        self._model = None
        self.name = name

    ## @brief Fits a Keras model
    # @param data Network Input
    # @param labels Network Output
    # @param epochs Number of training epochs
    # @param batch_size Size of training batches
    # @param validation_split Amount of data / labels to save for validation
    # @param callbacks Keras training callbacks
    def train(self, data: np.ndarray, labels: np.ndarray, epochs: int = 1, batch_size: int = 128,
              validation_split=0.2, callbacks=None):

        self._model.fit(data, labels, epochs=epochs, batch_size=batch_size,
                        validation_split=validation_split, callbacks=callbacks)

    ## @brief Predict a network output based on network input
    # @param samples Network input
    # @return Predicted network output
    def predict(self, samples) -> np.ndarray:
        return self._model.predict(samples)

    ## @brief Gets the shape of the network's input and output
    # @return Dict containing the keys 'input' and 'output' mapped to the network shape
    def shape(self) -> dict:
        return {'input': self._model.input_shape, 'output': self._model.output_shape}

    ## @brief Returns a string representation of the neural network
    # @return String representation of neural network
    def summary(self) -> str:
        return self._model.summary()

    ## @brief Loads a network from a saved weights file
    # @param path Path to the .h5 saved weights file
    # @return Boolean value representing call success
    def load(self, path: str) -> bool:
        return self._model.load_weights(path)

    ## @brief Saves network weights to a file
    # @param path Path to the .h5 weights file to save
    # @return Boolean value representing call success
    def save(self, path: str) -> bool:
        return self._model.save_weights(path)

    ## @brief Export and freezes the graph to a protobuff file
    # @param path Path to the .pb file
    # @return Boolean value representing call success
    def export(self, path: str) -> bool:
        import tensorflow as tf
        import keras.backend as K
        from tensorflow.python.framework import graph_io
        from tensorflow.python.tools import freeze_graph
        from tensorflow.core.protobuf import saver_pb2
        from tensorflow.python.training import saver as saver_lib

        with tf.Session() as sess:
            checkpoints_path = './checkpoints'

            sess.run(tf.global_variables_initializer())

            K.set_learning_phase(0)
            sess = K.get_session()
            saver = saver_lib.Saver(write_version=saver_pb2.SaverDef.V2)

            if not os.path.exists(checkpoints_path):
                os.mkdir(checkpoints_path)

            checkpoint_path = saver.save(sess, './checkpoints/%s.ckpt' % self.name, global_step=0)
            graph_io.write_graph(sess.graph, '.', 'tmp.pb')

            out_names = [node.name for node in tf.get_default_graph().as_graph_def().node]

            ret = freeze_graph.freeze_graph('./tmp.pb', '',
                                            False, checkpoint_path, ','.join(out_names),
                                            "save/restore_all", "save/Const:0",
                                            path, False, "")

            os.unlink('tmp.pb')
            return ret
        

class MicroFeatureMapModel(DNNModel):
    INPUT_CHANNELS = 1
    INPUT_RESOLUTION_X = 40
    INPUT_RESOLUTION_Y = 30
    CONV_DIM = 2
    
    INPUT_SHAPE = (int(INPUT_RESOLUTION_Y), int(INPUT_RESOLUTION_X), INPUT_CHANNELS)

    def __init__(self):

        DNNModel.__init__(self, name='MicroFeatureMapModel')
        from keras.layers import SeparableConv2D, Input, BatchNormalization, Conv2D, Activation, Dense, MaxPooling2D, AveragePooling2D, Flatten, concatenate, Reshape
        from keras.models import Model
        from keras.backend import set_session

        def mobilenet_layer(input_layer, strides, dim, batchnorm=False):
            mn_layer = input_layer
            mn_layer = SeparableConv2D(dim, strides)(mn_layer)
            if batchnorm:
                mn_layer = BatchNormalization()(mn_layer)
            mn_layer = Activation('relu')(mn_layer)
            mn_layer = Conv2D(dim * 2, (1, 1))(mn_layer)
            if batchnorm:
                mn_layer = BatchNormalization()(mn_layer)
            mn_layer = Activation('relu')(mn_layer)
            return mn_layer

        branch_1_input = Input(shape=MicroFeatureMapModel.INPUT_SHAPE)
        branch_2_input = Input(shape=MicroFeatureMapModel.INPUT_SHAPE)

        # Large Features
        branch_1 = branch_1_input
        branch_1 = mobilenet_layer(branch_1, (6, 6), np.power(MicroFeatureMapModel.CONV_DIM, 1))
        branch_1 = mobilenet_layer(branch_1, (4, 4), np.power(MicroFeatureMapModel.CONV_DIM, 2))
        branch_1 = mobilenet_layer(branch_1, (2, 2), np.power(MicroFeatureMapModel.CONV_DIM, 3))
        
        # Small Features
        branch_2 = branch_2_input
        branch_2 = mobilenet_layer(branch_2, (2, 2), np.power(MicroFeatureMapModel.CONV_DIM, 1))
        branch_2 = mobilenet_layer(branch_2, (4, 4), np.power(MicroFeatureMapModel.CONV_DIM, 2))
        branch_2 = mobilenet_layer(branch_2, (6, 6), np.power(MicroFeatureMapModel.CONV_DIM, 3))

        branch_out = concatenate([branch_1, branch_2])
        branch_out = Flatten()(branch_out)

        # Dimensionality Reduction / Heat Map
        branch_out = Dense(1)(branch_out)

        model = Model(inputs=[branch_1_input, branch_2_input], outputs=[branch_out])
        model.compile(optimizer='adam', loss='mse', metrics=['mse'])

        self._model = model

        import tensorflow as tf

        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        set_session(tf.Session(config=config))

    ## @brief Fits a Keras model
    # @param data Network Input
    # @param labels Network Output
    # @param epochs Number of training epochs
    # @param batch_size Size of training batches
    # @param validation_split Amount of data / labels to save for validation
    def train(self, data: np.ndarray, labels: np.ndarray, epochs: int = 1, batch_size: int = 128,
              validation_split=0.2, callbacks=None):

        from keras.callbacks import ModelCheckpoint

        filepath = "weights-{epoch:02d}-{val_loss:.4f}.hdf5"
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

        DNNModel.train(self, data, labels, epochs, batch_size, validation_split, [checkpoint])

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
print("Loading Data")
training_data = json.loads(open('training.json', 'r').read())

samples = training_data['samples']
labels = training_data['labels']

print("Converting to ndarrays")
samples = np.array(samples)
print(samples.shape)
samples = samples.reshape(samples.shape[0], samples.shape[2], samples.shape[1], samples.shape[3])
print(samples.shape)
labels = np.array(labels)

print("Done")

Loading Data
Converting to ndarrays
(63432, 40, 30, 1)
(63432, 30, 40, 1)
Done


In [4]:
# Training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    print("Training")
    model = MicroFeatureMapModel()
    model.train([samples, samples], labels, epochs=10)



Training
Train on 50745 samples, validate on 12687 samples
Epoch 1/10
50745/50745 [==============================] - 27s 530us/step - loss: 0.0102 - mean_squared_error: 0.0102 - val_loss: 0.0013 - val_mean_squared_error: 0.0013

Epoch 00001: val_loss improved from inf to 0.00128, saving model to weights-01-0.00.hdf5
Epoch 2/10
50745/50745 [==============================] - 26s 503us/step - loss: 0.0081 - mean_squared_error: 0.0081 - val_loss: 0.0020 - val_mean_squared_error: 0.0020

Epoch 00002: val_loss did not improve from 0.00128
Epoch 3/10
50745/50745 [==============================] - 26s 507us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0040 - val_mean_squared_error: 0.0040

Epoch 00003: val_loss did not improve from 0.00128
Epoch 4/10
50745/50745 [==============================] - 26s 509us/step - loss: 0.0070 - mean_squared_error: 0.0070 - val_loss: 0.0050 - val_mean_squared_error: 0.0050

Epoch 00004: val_loss did not improve from 0.00128
Epoch 5/10
50745/50

In [2]:
# Export
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    model = MicroFeatureMapModel()

    print("Exporting Graph")
    
    # Export Graph
    model.load('weights-01-0.00.hdf5')
    model.export('graph.pb')
    
    print("Done")

Using TensorFlow backend.


Exporting Graph
INFO:tensorflow:Restoring parameters from ./checkpoints/MicroFeatureMapModel.ckpt-0
INFO:tensorflow:Froze 37 variables.
Converted 37 variables to const ops.
Done


In [2]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    model = MicroFeatureMapModel()

    print(model.summary())

Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30, 40, 1)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 30, 40, 1)    0                                            
__________________________________________________________________________________________________
separable_conv2d_1 (SeparableCo (None, 25, 35, 2)    40          input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d_4 (SeparableCo (None, 29, 39, 2)    8           input_2[0][0]                    
__________________________________________________________________________________________________
activation

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    model = MicroFeatureMapModel()
    model.load('weights-01-0.00.hdf5')

    from time import time

    i = np.zeros((16*16, 30, 40, 1))
    model.predict([i, i])

    t_i = time()
    model.predict([i, i])
    t_f = time()
                  
    print(t_f - t_i)

0.02673816680908203
